In [1]:
import ipyvuetify as v
from utils import utils

In [2]:
#output 
foreground_connectivity = None
edge_width = None
transition_core = True
separate_feature = True
statistics = True

In [3]:
MSPA_MESSAGE = 'click on "Run process" to launch the process on your GEE account'

In [4]:
mspa_process_output = v.Alert(children=[MSPA_MESSAGE], type='info', text=True, class_="mt-5")

In [5]:
#select connectivity
connectivity = [4, 8] #using text input to transform it into 'confXX'
mspa_connectivity = v.Select(items=connectivity, label='Select the foreground connectivity', v_model=None)

def on_connectivity_change(widget, event, data):
    global foreground_connectivity
    foreground_connectivity = widget.v_model

mspa_connectivity.on_event('change', on_connectivity_change )

In [6]:
#select the edges
mspa_edge_slider = v.Slider(label= 'Threshold', class_="mt-5", thumb_label='always', v_model=0)

def on_edge_change(widget, event, data):
    global edge_width

    edge_width = widget.v_model

mspa_edge_slider.on_event('change', on_edge_change )

In [7]:
mspa_transition = v.Switch(label="Transition Core - Loop/Bridge", v_model=True)

def on_transition_change(widget, event, data):
    global transition_core

    transition_core = not transition_core

mspa_transition.on_event('change', on_transition_change )

In [8]:
mspa_feature = v.Switch(label="Separate internal from external features", v_model=True)

def on_feature_change(widget, event, data):
    global separate_feature

    separate_feature = not separate_feature

mspa_feature.on_event('change', on_feature_change )

In [9]:
mspa_statistics = v.Switch(label="Compute statistics", v_model=True)

def on_statistics_change(widget, event, data):
    global statistics

    statistics = not statistics

mspa_statistics.on_event('change', on_statistics_change )

In [10]:
#validate the selected data
mspa_process_btn = v.Btn(
    color='primary', 
    children=[
        v.Icon(left=True, children=['mdi-cogs']),
        'Run process'
    ]
)

def mspa_on_click(widget, event, data):
    
    global gfc_process_output
    global asset_name
    global threshold
    global gfc_process_btn
    
    #check for inputs
    
    #do something 
    utils.toggleLoading(gee_process_btn)
    mspa_process_output.children=['bouton cliqué']
    utils.toggleLoading(gee_process_btn)

mspa_process_btn.on_event('click', mspa_on_click)

In [11]:
#create the display 
inputs = v.Layout(
    _metadata={'mount-id': 'data-input'},
    row=True,
    class_="pa-5",
    align_center=True, 
    children=[
        v.Flex(xs12=True, children=[mspa_connectivity]),
        v.Flex(xs12=True, children=[mspa_edge_slider]),
        v.Flex(xs4=True, children=[mspa_transition]),
        v.Flex(xs4=True, children=[mspa_feature]),
        v.Flex(xs4=True, children=[mspa_statistics]),
        v.Flex(xs12=True, children=[mspa_process_btn]),
        v.Flex(xs12=True, children=[mspa_process_output]),
    ]
)
#inputs

MSPA_content_main =  v.Layout(
    _metadata={'mount_id': 'mspa_process_widget'},
    row=True,
    xs12=True,
    align_center=True, 
    class_="ma-5 d-inline",
    children=[
        v.Card( 
            class_="pa-5",
            raised=True,
            xs12=True,
            children=[
                v.Html(xs12=True, tag='h2', children=['Step 3: Run the map fragmentation']),
                v.Flex(xs12=True, lg6=True, children=[inputs]),   
            ]
        )
    ]
)

MSPA_content_main

Layout(align_center=True, children=[Card(children=[Html(children=['Step 3: Run the map fragmentation'], tag='h…